In [46]:
import gdown
url = "https://drive.google.com/uc?id=1a2Hsbf0k1fSbKAYYVp00YF3i_OtWE31j"  # convert sharing link to direct download
output = "merged_data.csv"
gdown.download(url, output, quiet=False)

import pandas as pd
data = pd.read_csv(output)

data.head()

Downloading...
From: https://drive.google.com/uc?id=1a2Hsbf0k1fSbKAYYVp00YF3i_OtWE31j
To: /content/merged_data.csv
100%|██████████| 31.2M/31.2M [00:00<00:00, 64.8MB/s]


,c_id,application_hour,application_day_of_week,account_open_year,preferred_contact,num_login_sessions,num_customer_service_calls,has_mobile_app,paperless_billing,default,...,employment_type,education,marital_status,num_dependents,credit_score,num_credit_accounts,oldest_credit_line_age,oldest_account_age_months,total_credit_limit,account_diversity_index
0,10000,5,6,2013,Mail,13,2,1,1,0,...,Full-time,Graduate,Married,2,696,14,22.8,273.6,169100,0.499
1,10001,4,2,2015,Phone,6,1,1,1,1,...,FULL_TIME,High School,Married,0,659,13,3.5,42.0,78200,0.298
2,10002,10,3,2020,Phone,1,2,1,0,0,...,FULL_TIME,Bachelor,Single,0,662,3,0.0,0.0,41400,0.174
3,10003,7,5,2010,Email,4,1,1,1,0,...,Full Time,Bachelor,Single,0,676,8,9.0,108.0,60000,0.263
4,10004,1,2,2020,Mail,6,2,1,0,0,...,Fulltime,High School,Single,0,678,7,8.0,96.0,49700,0.298


### Missing Values Check

### Checking

In [47]:
import pandas as pd

missing_counts = data.isnull().sum()
missing_percent = (missing_counts / len(data)) * 100

missing_df = pd.DataFrame({'MissingCount': missing_counts,
    'MissingPercent': missing_percent}).sort_values(by='MissingCount', ascending=False)

print(missing_df)


                            MissingCount  MissingPercent
employment_length                   2253        2.503361
revolving_balance                   1377        1.530017
application_hour                       0        0.000000
application_day_of_week                0        0.000000
preferred_contact                      0        0.000000
account_open_year                      0        0.000000
num_customer_service_calls             0        0.000000
has_mobile_app                         0        0.000000
paperless_billing                      0        0.000000
num_login_sessions                     0        0.000000
default                                0        0.000000
monthly_income                         0        0.000000
monthly_payment                        0        0.000000
existing_monthly_debt                  0        0.000000
debt_to_income_ratio                   0        0.000000
debt_service_ratio                     0        0.000000
payment_to_income_ratio        

We can see that there are missing values in employment_length and revolving_balance

In [48]:
data['revolving_balance'] = data['revolving_balance'].replace('[\$,]', '', regex=True).astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-2299545015.py:1: SyntaxWarning: invalid escape sequence '\$'
  data['revolving_balance'] = data['revolving_balance'].replace('[\$,]', '', regex=True).astype(float)


### Imputing

In [49]:
cols = ['employment_length', 'revolving_balance']

for col in cols:
  mean_val = data[col].mean()
  median_val = data[col].median()
  mode_val = data[col].mode()[0]  # mode() returns a Series

  print(f"{col}:")
  print(f"  Mean   = {mean_val}")
  print(f"  Median = {median_val}")
  print(f"  Mode   = {mode_val}\n")

employment_length:
  Mean   = 5.514892986574887
  Median = 5.2
  Mode   = 4.9

revolving_balance:
  Mean   = 39561.29588364063
  Median = 29294.7
  Mode   = 18468.0



Median is a robust choice for both

In [50]:
from sklearn.impute import SimpleImputer

emp_imputer = SimpleImputer(strategy='median')
data[['employment_length']] = emp_imputer.fit_transform(data[['employment_length']])

rev_imputer = SimpleImputer(strategy='median')
data[['revolving_balance']] = rev_imputer.fit_transform(data[['revolving_balance']])

### Type Conversion where needed

In [51]:
print(data.dtypes)

c_id                            int64
application_hour                int64
application_day_of_week         int64
account_open_year               int64
preferred_contact              object
num_login_sessions              int64
num_customer_service_calls      int64
has_mobile_app                  int64
paperless_billing               int64
default                         int64
monthly_income                 object
existing_monthly_debt          object
monthly_payment                object
debt_to_income_ratio          float64
debt_service_ratio            float64
payment_to_income_ratio       float64
credit_utilization            float64
revolving_balance             float64
credit_usage_amount            object
available_credit               object
total_monthly_debt_payment     object
annual_debt_payment           float64
loan_to_annual_income         float64
total_debt_amount              object
monthly_free_cash_flow         object
regional_unemployment_rate    float64
regional_med

In [52]:
# Convert numeric columns stored as objects to float
numeric_obj_cols = [
    'monthly_income', 'existing_monthly_debt', 'monthly_payment', 'credit_usage_amount',
        'available_credit', 'total_monthly_debt_payment', 'total_debt_amount',
            'monthly_free_cash_flow', 'loan_amount', 'annual_income']

for col in numeric_obj_cols:
  data[col] = data[col].replace('[\$,]', '', regex=True).astype(float)

categorical_cols = ['preferred_contact', 'loan_type', 'loan_purpose', 'origination_channel',
                            'employment_type', 'education', 'marital_status']

for col in categorical_cols:
  data[col] = data[col].astype('category')

print(data.dtypes)

<>:8: SyntaxWarning: invalid escape sequence '\$'
<>:8: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-737979403.py:8: SyntaxWarning: invalid escape sequence '\$'
  data[col] = data[col].replace('[\$,]', '', regex=True).astype(float)


c_id                             int64
application_hour                 int64
application_day_of_week          int64
account_open_year                int64
preferred_contact             category
num_login_sessions               int64
num_customer_service_calls       int64
has_mobile_app                   int64
paperless_billing                int64
default                          int64
monthly_income                 float64
existing_monthly_debt          float64
monthly_payment                float64
debt_to_income_ratio           float64
debt_service_ratio             float64
payment_to_income_ratio        float64
credit_utilization             float64
revolving_balance              float64
credit_usage_amount            float64
available_credit               float64
total_monthly_debt_payment     float64
annual_debt_payment            float64
loan_to_annual_income          float64
total_debt_amount              float64
monthly_free_cash_flow         float64
regional_unemployment_rat

### Encoding Categorical Features

In [40]:
categorical_cols = data.select_dtypes(include='category').columns.tolist()
print(categorical_cols)

['preferred_contact', 'loan_type', 'loan_purpose', 'origination_channel', 'employment_type', 'education', 'marital_status']


In [53]:
data['loan_type_original'] = data['loan_type'].copy()
data['employment_type_original'] = data['employment_type'].copy()

In [54]:
data['loan_type_clean'] = data['loan_type'].str.strip().str.lower()
data['employment_type_clean'] = data['employment_type'].str.strip().str.lower()

In [55]:
loan_type_map = {
      'personal': 'Personal',
          'personal loan': 'Personal',
              'personalloan': 'Personal',
                  'cc': 'Credit Card',
                      'credit card': 'Credit Card',
                          'creditcard': 'Credit Card',
                              'mortgage': 'Mortgage',
                                  'mortgage loan': 'Mortgage',
                                      'home loan': 'Home Loan'
                                      }

employment_type_map = {
      'full-time': 'Full-time',
          'full time': 'Full-time',
              'full_time': 'Full-time',
                  'fulltime': 'Full-time',
                      'ft': 'Full-time',
                          'part time': 'Part-time',
                              'part-time': 'Part-time',
                                  'pt': 'Part-time',
                                      'self employed': 'Self-employed',
                                          'self emp': 'Self-employed',
                                              'self-employed': 'Self-employed',
                                                  'self_employed': 'Self-employed',
                                                      'contract': 'Contract',
                                                          'contractor': 'Contract'
                                                          }
data['loan_type'] = data['loan_type_clean'].replace(loan_type_map)
data['employment_type'] = data['employment_type_clean'].replace(employment_type_map)

# Optional: drop temporary clean columns
data.drop(['loan_type_clean', 'employment_type_clean'], axis=1, inplace=True)

# Verify results
print("Loan type categories:", data['loan_type'].unique())
print("Employment type categories:", data['employment_type'].unique())
print("Missing values in loan_type:", data['loan_type'].isnull().sum())
print("Missing values in employment_type:", data['employment_type'].isnull().sum())

Loan type categories: ['Personal' 'Mortgage' 'Credit Card' 'Home Loan']
Employment type categories: ['Full-time' 'Part-time' 'Self-employed' 'Contract' 'part_time']
Missing values in loan_type: 0
Missing values in employment_type: 0


In [57]:
categorical_cols = ['loan_type', 'employment_type', 'preferred_contact',
                    'loan_purpose', 'origination_channel', 'education', 'marital_status']

for col in categorical_cols:
  uniques = data[col].dropna().unique()  # dropna to ignore missing values
  print(f"{col} unique values ({len(uniques)}): {uniques}\n")


loan_type unique values (4): ['Personal' 'Mortgage' 'Credit Card' 'Home Loan']

employment_type unique values (5): ['Full-time' 'Part-time' 'Self-employed' 'Contract' 'part_time']

preferred_contact unique values (3): ['Mail', 'Phone', 'Email']
Categories (3, object): ['Email', 'Mail', 'Phone']

loan_purpose unique values (8): ['Debt Consolidation', 'Refinance', 'Major Purchase', 'Medical', 'Revolving Credit', 'Home Improvement', 'Home Purchase', 'Other']
Categories (8, object): ['Debt Consolidation', 'Home Improvement', 'Home Purchase', 'Major Purchase',
                         'Medical', 'Other', 'Refinance', 'Revolving Credit']

origination_channel unique values (4): ['Direct Mail', 'Branch', 'Online', 'Broker']
Categories (4, object): ['Branch', 'Broker', 'Direct Mail', 'Online']

education unique values (5): ['Graduate', 'High School', 'Bachelor', 'Some College', 'Advanced']
Categories (5, object): ['Advanced', 'Bachelor', 'Graduate', 'High School', 'Some College']

marital_statu

In [58]:
data['employment_type'] = data['employment_type'].replace({'part_time': 'Part-time'})

In [59]:
print(data['employment_type'].unique())

['Full-time' 'Part-time' 'Self-employed' 'Contract']


In [61]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

ordinal_cols = ['education']  # Columns with natural order
nominal_cols = ['loan_type', 'employment_type', 'loan_purpose',
                'preferred_contact', 'origination_channel', 'marital_status']

education_order = ["High School", "Some College", "Bachelor", "Graduate", "Advanced"]
data['education'] = pd.Categorical(data['education'], categories=education_order, ordered=True)
data['education_encoded'] = data['education'].cat.codes

data = pd.get_dummies(data, columns=nominal_cols, drop_first=True)

print("Encoded columns sample:")
print(data.head())
print("\nAll columns:")
print(data.columns)

Encoded columns sample:
    c_id  application_hour  application_day_of_week  account_open_year  \
0  10000                 5                        6               2013   
1  10001                 4                        2               2015   
2  10002                10                        3               2020   
3  10003                 7                        5               2010   
4  10004                 1                        2               2020   

   num_login_sessions  num_customer_service_calls  has_mobile_app  \
0                  13                           2               1   
1                   6                           1               1   
2                   1                           2               1   
3                   4                           1               1   
4                   6                           2               1   

   paperless_billing  default  monthly_income  ...  loan_purpose_Other  \
0                  1        0         5150

In [62]:
cols_to_drop = ['loan_type_original', 'employment_type_original', 'education']
data.drop(columns=cols_to_drop, inplace=True)

In [63]:
print(data.dtypes)

c_id                               int64
application_hour                   int64
application_day_of_week            int64
account_open_year                  int64
num_login_sessions                 int64
                                   ...  
origination_channel_Broker          bool
origination_channel_Direct Mail     bool
origination_channel_Online          bool
marital_status_Married              bool
marital_status_Single               bool
Length: 64, dtype: object


## Scaling and transformations

In [67]:
continuous_cols = [ 'monthly_income', 'existing_monthly_debt', 'monthly_payment',
                   'debt_to_income_ratio', 'revolving_balance', 'credit_usage_amount',
                    'available_credit', 'total_monthly_debt_payment', 'annual_debt_payment',
                     'loan_to_annual_income', 'total_debt_amount', 'monthly_free_cash_flow',
                     'loan_amount', 'interest_rate', 'loan_to_value_ratio', 'age',
                    'annual_income', 'employment_length', 'num_dependents', 'credit_score',
                    'num_credit_accounts', 'oldest_credit_line_age', 'oldest_account_age_months',
                    'total_credit_limit', 'account_diversity_index']
print(data[continuous_cols].describe())
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.max_colwidth', None)  # Full string width
print(data[continuous_cols].skew().sort_values(ascending=False))

       monthly_income  existing_monthly_debt  monthly_payment  \
count    89999.000000           89999.000000     89999.000000   
mean      3965.571882             894.136941       927.723597   
std       2224.252044             679.000900       892.264464   
min       1666.670000              83.360000        15.000000   
25%       2341.670000             424.475000       294.000000   
50%       3425.000000             706.150000       565.940000   
75%       4941.670000            1168.840000      1325.955000   
max      40600.000000            9229.850000      7542.690000   

       debt_to_income_ratio  revolving_balance  credit_usage_amount  \
count          89999.000000       89999.000000         89999.000000   
mean               0.531225       39404.215221         39557.125978   
std                0.379553       34522.296287         34753.792901   
min                0.054000         804.300000           804.300000   
25%                0.296000       16731.600000         1654

In [68]:
from sklearn.preprocessing import RobustScaler

# List of continuous features
continuous_cols = [ 'monthly_income', 'existing_monthly_debt', 'monthly_payment',
                   'debt_to_income_ratio', 'revolving_balance', 'credit_usage_amount',
                    'available_credit', 'total_monthly_debt_payment', 'annual_debt_payment',
                     'loan_to_annual_income', 'total_debt_amount', 'monthly_free_cash_flow',
                     'loan_amount', 'interest_rate', 'loan_to_value_ratio', 'age',
                      'annual_income', 'employment_length', 'num_dependents', 'credit_score',
                        'num_credit_accounts', 'oldest_credit_line_age', 'oldest_account_age_months',
                     'total_credit_limit', 'account_diversity_index']

scaler = RobustScaler()

# Fit and transform the continuous features
data[continuous_cols] = scaler.fit_transform(data[continuous_cols])

# Verify the scaling
print(data[continuous_cols].describe().T)

                              count      mean       std       min       25%  \
monthly_income              89999.0  0.207912  0.855482 -0.676281 -0.416665   
existing_monthly_debt       89999.0  0.252547  0.912188 -0.836673 -0.378410   
monthly_payment             89999.0  0.350581  0.864635 -0.533880 -0.263519   
debt_to_income_ratio        89999.0  0.322968  1.143232 -1.114458 -0.385542   
revolving_balance           89999.0  0.299685  1.023374 -0.844565 -0.372419   
credit_usage_amount         89999.0  0.298858  1.010675 -0.828114 -0.370216   
available_credit            89999.0  0.296130  1.012446 -0.861956 -0.372184   
total_monthly_debt_payment  89999.0  0.184203  0.762226 -0.994920 -0.395885   
annual_debt_payment         89999.0  0.184203  0.762226 -0.994920 -0.395885   
loan_to_annual_income       89999.0  0.625846  1.224447 -0.134630 -0.077995   
total_debt_amount           89999.0  0.396136  0.771617 -0.331929 -0.157810   
monthly_free_cash_flow      89999.0  0.137494  0.935

In [69]:
print(data.head())
print(data.dtypes)
print(data.isnull().sum())
print(len(data.columns))

    c_id  application_hour  application_day_of_week  account_open_year  \
0  10000                 5                        6               2013   
1  10001                 4                        2               2015   
2  10002                10                        3               2020   
3  10003                 7                        5               2010   
4  10004                 1                        2               2020   

   num_login_sessions  num_customer_service_calls  has_mobile_app  \
0                  13                           2               1   
1                   6                           1               1   
2                   1                           2               1   
3                   4                           1               1   
4                   6                           2               1   

   paperless_billing  default  monthly_income  existing_monthly_debt  \
0                  1        0        0.663462               0.043648

In [70]:
data.to_csv("preprocessed_credit_data.csv", index=False)